# setup django env

## db server

## environment

# confirm apis

## crop pic by skimage?

# retrieve

## nmslib 
这是一个大问题，需要更多的算法，做更好的比较
## plot nmslib result on x-y plot

# aligner
初步认为是因为人脸没有对齐，导致索引极差。但这个事情应分析下facenet的训练要求才能解决

# f1 score


# 想法
把这几个识别库单独拿出来，做成一个开发包，与Django和Celery分开使用

In [3]:
import numpy as np
import pandas as pd
import sys
import os

In [6]:
import sys
import os
import django
def start_local():
    os.environ["DOCKER_MODE"] = "true"
    os.environ["RABBIT_HOST"]= "localhost"
    os.environ["RABBIT_USER"] = "dvauser"
    os.environ["RABBIT_PASS"] = "localpass"

    # db
    os.environ["DB_HOST"] = "localhost"
    os.environ["DB_NAME"] = "postgres"
    os.environ["DB_USER"] = "pgdbuser"
    os.environ["DB_PASS"] = "pgdbpass"

    os.environ["LAUNCH_SERVER"] = "1"
    os.environ["LAUNCH_BY_NAME_indexer_inception"] = "1"
    os.environ["LAUNCH_BY_NAME_indexer_facenet"] = "1"
    os.environ["LAUNCH_BY_NAME_retriever_inception"] = "1"
    os.environ["LAUNCH_BY_NAME_retriever_facenet"] = "1"
    os.environ["LAUNCH_BY_NAME_detector_coco"] = "1"
    os.environ["LAUNCH_BY_NAME_analyzer_crnn"] = "1"
    os.environ["LAUNCH_BY_NAME_analyzer_tagger"] = "1"
    os.environ["LAUNCH_BY_NAME_detector_face"] = "1"
    os.environ["LAUNCH_Q_qclusterer"] = "1"
    os.environ["LAUNCH_Q_qextract"] = "1"
    os.environ["LAUNCH_SCHEDULER"] = "1"
    # export  TEST=1
    os.environ["AUTH_DISABLED"] = "1"
    dva_home = "/home/tom/ai/DeepVideoAnalytics"
    sys.path.append(os.path.abspath(dva_home))
    
    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
    django.setup()
    
start_local()

In [5]:
bmp = "/home/tom/ai.dataset.images/Face/222/222_0.bmp"
temppath = "/home/tom/ai.dataset.images.tmp"

In [4]:
import dvalib.indexer
index = dvalib.indexer.FacenetIndexer(os.path.join("/home/tom/ai/DeepVideoAnalytics/dva/media/indexers/2", "facenet.pb"))
index.load()

In [6]:
from dvalib import detector
reload(detector)
detect =detector.FaceDetector()
detect.load()

Using TensorFlow backend.


In [7]:
from dvalib.dlib import AlignDlib
aligner = AlignDlib(os.path.join("/home/tom/ai/models/dlib","shape_predictor_68_face_landmarks.dat"))

In [9]:
from dvalib import retriever
args = {}
args["components"] =16 # pca component
args["m"] = 4 # totol number of fine codes
args["v"] = 8 # number of cluster per coarse quntizer
args["sub"] = 256 # the number of cluster to train per subquantizer
args['proto_filename'] = os.path.join(temppath, "retriever.proto")
args['proto_filename'].replace('.proto','.P.npy')
args['proto_filename'].replace('.proto','.mu.npy')
args['proto_filename'].replace('.proto', '.pca.pkl')
args['proto_filename'].replace('.proto', '_lmdb')
args['proto_filename'].replace('.proto', '.permuted_inds.pkl')

lopq = retriever.LOPQRetriever("lopq", args)

In [8]:
import skimage.io as io
import os.path
from scipy import misc
if not os.path.exists(temppath):
    os.mkdir(temppath)
basename = os.path.basename(bmp)
dirname  = os.path.dirname(bmp)
png = os.path.join(temppath, basename.split(".")[0] + ".png")
image = io.imread(bmp)
io.imsave(png, image)
#misc.imshow(image)

import scipy
from scipy import misc
crop = detect.cropLargest(misc.imread(png))
#misc.imshow(crop)

al = aligner.align(96, crop)
#misc.imshow(al)

cropfile = os.path.join(temppath, (basename.split('.')[0]+ "crop"+".png"))
io.imsave(cropfile, al)
#detect.cropLargestToFile(png, cropfile)

misc.imshow(al)

AttributeError: FaceDetector instance has no attribute 'cropLargest'

In [22]:
feat      = np.squeeze(index.apply(png))
feat_head = np.squeeze(index.apply(png))

print("dot(feat,feat)={}".format(np.dot(feat,feat)))
print("dot(head, head)={}".format(np.dot(feat_head, feat_head)))
print("dot(feat, head)={}".format(np.dot(feat, feat_head)))

FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,160,160,3], <unknown>], output_types=[DT_FLOAT, DT_STRING], _device="/job:localhost/replica:0/task:0/cpu:0"](Iterator)]]

Caused by op u'IteratorGetNext', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  F

In [27]:
print("image_path={}".format(png))
index.apply(png)

image_path=/home/tom/ai.dataset.images.tmp/222_0.png


FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element.
	 [[Node: IteratorGetNext = IteratorGetNext[output_shapes=[[?,160,160,3], <unknown>], output_types=[DT_FLOAT, DT_STRING], _device="/job:localhost/replica:0/task:0/cpu:0"](Iterator)]]

Caused by op u'IteratorGetNext', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  F

In [44]:
import skimage.io as io
import collections
PicRec = collections.namedtuple("PicRec",["path","feat"])
def detect_dir(path, dest_path) :
    result = []
    faces = os.listdir(path)
    for f in faces:
        picpath = os.path.join(path, f)
        dest_file = os.path.join(dest_path,f)
        hasface = detect.cropLargestToFile(picpath, dest_file)
        
        if hasface:
            feat = np.squeeze(index.apply(dest_file))
            result.append(PicRec(path=f.split("_")[0], feat=feat))
        else :
            print("fail to detect face in {}".format(path))
    return result
dest_dir = "/home/tom/ai.dataset.images.tmp/test.temp"
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)
r = detect_dir("/home/tom/ai.dataset.images.tmp/test", dest_dir)
np.dot(r[0].feat, r[1].feat)

0.99999988

In [45]:
face_dir = "/home/tom/ai.dataset.images.tmp/Face_DB_4_ms"
test_dir = "/home/tom/ai.dataset.images.tmp/Face_Test_ms"
crop_face_dir = "/home/tom/ai.dataset.images.tmp/Face_DB_4_ms_crop"
crop_test_dir = "/home/tom/ai.dataset.images.tmp/Face_Test_ms_crop"

if not os.path.exists(crop_face_dir) :
    os.mkdir(crop_face_dir)
if not os.path.exists(crop_test_dir) :
    os.mkdir(crop_test_dir)
    
#os.mkdir(crop_test_dir)
valid = detect_dir(test_dir, crop_test_dir)
train = detect_dir(face_dir, crop_face_dir)

fail to detect face in /home/tom/ai.dataset.images.tmp/Face_DB_4_ms
fail to detect face in /home/tom/ai.dataset.images.tmp/Face_DB_4_ms


In [57]:
def save_feat(data, path):
    feats = [ d.feat for d in data ]
    arr = np.array(feats)
    print(arr.shape)
    np.save(path, arr)
    
save_feat(train, "train.npy")
save_feat(valid, "valid.npy")
    

(1598, 128)
(501, 128)


In [46]:
import os.path
import nmslib 
import numpy as np

INDEX_DIR='indices'    
import shutil
if os.path.exists(INDEX_DIR):
    shutil.rmtree(INDEX_DIR)

class NmslibReuseIndex:
    def __init__(self, metric, method_name, index_param, save_index, query_param):
        self._nmslib_metric = {'angular': 'cosinesimil', 'euclidean': 'l2'}[metric]
        self._method_name = method_name
        self._save_index = save_index
        self._index_param = index_param
        self._query_param = query_param
        self.name = 'Nmslib(method_name=%s, index_param=%s, query_param=%s)' % (
            method_name, index_param, query_param)
        
        self._index_name = os.path.join(INDEX_DIR, "nmslib_%s_%s_%s" % (
            self._method_name, metric, '_'.join(self._index_param))) 

        d = os.path.dirname(self._index_name)
        if not os.path.exists(d):
          os.makedirs(d)
        
        self._index = nmslib.init(self._nmslib_metric)

    def add(self, i, x) :
        self._index.addDataPoint(i,x)
        
    def fit(self, X):
        if self._method_name == 'vptree':
            # To avoid this issue:
            # terminate called after throwing an instance of 'std::runtime_error'
            # what():  The data size is too small or the bucket size is too big. Select the parameters so that <total # of records> is NOT less than <bucket size> * 1000
            # Aborted (core dumped)
            self._index_param.append('bucketSize=%d' % min(int(X.shape[0] * 0.0005), 1000))
                                        
        for i, x in enumerate(X):
            self._index.addDataPoint(i, x)


        if os.path.exists(self._index_name):
            print "Loading index from file"
            self._index.loadIndex(self._index_name)
        else:
            self._index.createIndex(self._index_param)
            if self._save_index: 
              self._index.saveIndex(self._index_name)
        #self._index.setQueryTimeParams(self._query_param)
        

    def query(self, v, n):
        return self._index.knnQuery( v, k=n)

    def freeIndex(self):
        self._index = None
        
class FALCONN(object):
    def __init__(self, metric, num_bits, num_tables, num_probes):
        self.name = 'FALCONN(K={}, L={}, T={})'.format(num_bits, num_tables, num_probes)
        self._metric = metric
        self._num_bits = num_bits
        self._num_tables = num_tables
        self._num_probes = num_probes
        self._center = None
        self._params = None
        self._index = None
        self._buf = None

    def fit(self, X):
        if X.dtype != numpy.float32:
            X = X.astype(numpy.float32)
        if self._metric == 'angular':
            X /= numpy.linalg.norm(X, axis=1).reshape(-1,  1)
        self._center = numpy.mean(X, axis=0)
        X -= self._center
        import falconn
        self._params = falconn.LSHConstructionParameters()
        self._params.dimension = X.shape[1]
        self._params.distance_function = 'euclidean_squared'
        self._params.lsh_family = 'cross_polytope'
        falconn.compute_number_of_hash_functions(self._num_bits, self._params)
        self._params.l = self._num_tables
        self._params.num_rotations = 1
        self._params.num_setup_threads = 0
        self._params.storage_hash_table = 'flat_hash_table'
        self._params.seed = 95225714
        self._index = falconn.LSHIndex(self._params)
        self._index.setup(X)
        self._index.set_num_probes(self._num_probes)
        self._buf = numpy.zeros((X.shape[1],), dtype=numpy.float32)

    def query(self, v, n):
        numpy.copyto(self._buf, v)
        if self._metric == 'angular':
            self._buf /= numpy.linalg.norm(self._buf)
        self._buf -= self._center
        return self._index.find_k_nearest_neighbors(self._buf, n)

data = np.random.randn(100, 2).astype(np.float32)
index = NmslibReuseIndex("angular", "hnsw", {"post":2}, False, {"k":10})
index.fit(data)
index.query(data[0], 10)

(array([10,  0, 77,  6, 86, 65, 43, 50, 71, 38], dtype=int32),
 array([ 0.        ,  0.        ,  0.00017393,  0.00023609,  0.00036091,
         0.00076026,  0.00902432,  0.01111245,  0.01119417,  0.03453648], dtype=float32))

In [53]:
index = NmslibReuseIndex("euclidean", "hnsw", {"post":2}, False, {"k":10})
names = []
for i, (name,feat) in enumerate(train):
    index.add(i, feat)
    names.append(name)

index.fit([])

RecRec = collections.namedtuple("RecRec",["target","result", "similarity"])
query= []
for i, (name, feat) in enumerate(valid):
    r = index.query(feat, 1)
    if len(r[0]) > 0:
        rec = RecRec(target=name, result=names[r[0][0]], similarity=1.0-r[1][0])
    else:
        rec = RecRec(target=name, result="", similarity=0.0)
    query.append(rec)
    
face_test_f1_dir = "/home/tom/ai.dataset.images/face_test_f1"
fn = os.path.join(face_test_f1_dir, "facenet_test.csv")
with open(fn,"w") as f:
    f.write("target,result,similarity\n")
    for target, result, similarity in query:
        f.write("{},{},{}\n".format(target, result,similarity))
    

In [51]:
db_names_path = "db_names.txt"       
db_feats_path = "db_feats.txt"
def save_index(featpath, namepath, nf):
    with open(featpath,"w") as f:
        with open(namepath,"w") as fnames:
            for i, (name, feat) in enumerate(nf):
                fnames.write("{}\n".format(name))
                f.write("{},{}\n".format(name, ",".join([ str(featvalue) for featvalue in feat])))
save_index( "db_feats.txt", "db_names.txt", train)
save_index( "va_feats.txt", "va_names.txt", valid)